## 收集数据
- tweet无法访问 :)
- 从github仓库[https://github.com/udacity/new-dand-advanced-china](https://github.com/udacity/new-dand-advanced-china)中下载文件

In [ ]:
# -*- coding=utf-8 -*-
import requests

def download(url):
    with open(url.split('/')[-1], mode="wb") as f:
        respone = requests.get(url)
        f.write(respone.content)
        
# 下载 image-predictions.tsv
image_prediction_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv"
download(image_prediction_url)
# 下载 twitter_archive_enhanced.csv
twitter_archive_enhanced_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/twitter-archive-enhanced.csv"
download(twitter_archive_enhanced_url)
# 下载 tweet_json.txt
tweet_json_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/tweet_json.txt"
download(tweet_json_url)

## 评估数据

In [ ]:
# 使用pandas读取数据文件
import pandas as pd

In [ ]:
image_predictions = pd.read_csv("image-predictions.tsv", sep='\t')
image_predictions.info()

In [ ]:
image_predictions.head(3)

In [ ]:
# 处理和评估tweet_json.txt数据
import json
with open("tweet_json.txt", 'r') as json_file:
    for data in json_file.readlines():
        print(json.dumps(json.loads(data), indent=4, sort_keys=False, ensure_ascii=True))
        break

In [ ]:
# 逐行读取tweet_json.txt文件并添加到pandas DataFrame中，（至少）包含 tweet ID、retweet_count和favorite_count字段...
# 提出问题，放到list中，然后通过 'id_str': list1, 'fav_count': list2的方式创建df的效率是否更高？
df_tweet = pd.DataFrame()

# 观察tweet_json.txt后，对以下数据感兴趣：
# retweet_count favorite_count full_text retweeted source favorited
with open("tweet_json.txt", 'r') as json_file:
    index = ['id', 'retweet_count', 'favorite_count', 'full_text', 'retweeted', 'source', 'favorited']
    for data in json_file.readlines():
        dict_data = json.loads(data)
        
        data = []
        for idx in index:
            data.append(dict_data[idx])
        
        s_tweet = pd.Series(data, index=index)
        retweet_count = dict_data[u'retweet_count']
        df_tweet = df_tweet.append(s_tweet, ignore_index=True)

df_tweet.head(3)

In [ ]:
df_tweet.info()

In [ ]:
df_tweet.describe()

In [ ]:
df_tweet

In [ ]:
witter_archive_enhanced = pd.read_csv("twitter-archive-enhanced.csv")
witter_archive_enhanced.info()
witter_archive_enhanced.describe()

In [ ]:
witter_archive_enhanced

#### 质量
##### `df_tweet` 表格
- favorite_count、id、retweet_count应该是int64类型，而不是浮点浮点类型
- favorited 和 retweeted 的值都是零，需要删除这两列

##### `twitter_archive_enhanced` 表格
- doggo floofer pupper puppo字段有"None"这是python关键字，在csv体现为字符串，这应该在表示空
- in_reply_to_status_id和in_reply_to_user_id，只有78个值，这两个列的数据很可能没用
- timestamp、retweeted_status_timestamp现在是object类型，要转换为时间类型
- rating_denominator不是10的需要清理掉, 比如170是最大值是不对的
- rating_numerator分子小于rating_denominator的需要清理掉
- expanded_urls的单条记录中有重复的url地址，清理清理只保留一个
- name字段有字符串"None"，有的跟text中的描述不符，比如"a"

#### 清洁度
##### `df_tweet` 表格
- source字段内容是html文本，需要更加简洁，只需要获取该html的text部分即可
- 通过witter_archive_enhanced.tweet_id和df_tweet.id和image_predictions.tweet_id关联创建新的表格twitter_archive_master.csv

## 清理

In [ ]:
# 清理前的备份
df_tweet_clean = df_tweet.copy()
witter_archive_enhanced_clean = witter_archive_enhanced.copy()

In [ ]:
# favorite_count id retweet_count应该是int64类型，现在做格式转换 long
for i in ['favorite_count', 'id', 'retweet_count']:
    df_tweet_clean[i] = df_tweet_clean[i].astype(long)
df_tweet_clean.info()

In [ ]:
# 顺便调整列的顺序, 排除favorited和retweeted列
index = ['id', 'retweet_count', 'favorite_count', 'full_text', 'source']
df_tweet_clean = df_tweet_clean.reindex_axis(index, axis=1)
df_tweet_clean

In [ ]:
# rating_denominator不是10的需要清理掉
witter_archive_enhanced_clean = witter_archive_enhanced[witter_archive_enhanced["rating_denominator"]==10]
witter_archive_enhanced_clean.info()
witter_archive_enhanced_clean

In [ ]:
# rating_numerator分子小于rating_denominator的需要清理掉
witter_archive_enhanced_clean = witter_archive_enhanced_clean[witter_archive_enhanced_clean["rating_numerator"] > witter_archive_enhanced_clean["rating_denominator"]]
witter_archive_enhanced_clean.info()
witter_archive_enhanced_clean

In [ ]:
witter_archive_enhanced_clean.name.value_counts()

In [ ]:
# timestamp、retweeted_status_timestamp现在是object类型，要转换为时间类型
witter_archive_enhanced_clean['timestamp'] = pd.to_datetime(witter_archive_enhanced_clean['timestamp'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
witter_archive_enhanced_clean.info()

In [ ]:
# expanded_urls的单条记录中有重复的url地址，清理清理只保留一个
witter_archive_enhanced_clean['expanded_urls_one'], witter_archive_enhanced_clean['expanded_urls_others'] = witter_archive_enhanced_clean['expanded_urls'].str.split(',', 1).str
witter_archive_enhanced_clean

In [ ]:
# source字段内容是html文本，需要更加简洁，只需要获取该html的text部分即可
witter_archive_enhanced_clean['source1'], witter_archive_enhanced_clean['source2']= witter_archive_enhanced_clean.source.str.split('>', 1).str
witter_archive_enhanced_clean['source_text'], witter_archive_enhanced_clean['source4']= witter_archive_enhanced_clean.source2.str.split('<', 1).str
witter_archive_enhanced_clean = witter_archive_enhanced_clean.drop('source1', axis=1)
witter_archive_enhanced_clean = witter_archive_enhanced_clean.drop('source2', axis=1)
witter_archive_enhanced_clean = witter_archive_enhanced_clean.drop('source4', axis=1)
witter_archive_enhanced_clean

In [ ]:
# 查看source的分布
witter_archive_enhanced_clean.source_text.value_counts()

In [ ]:
# 修改df_tweet_clean的 id 为 tweet_id
df_tweet_clean['tweet_id'] = df_tweet_clean['id']
# 只保留部分列
index = ['tweet_id', 'retweet_count', 'favorite_count']
df_tweet_clean = df_tweet_clean.reindex_axis(index, axis=1)
index = ['tweet_id', 'name','rating_numerator','rating_denominator','timestamp', 'text', 'source_text', 'expanded_urls_one','doggo','floofer','pupper','puppo']
witter_archive_enhanced_clean = witter_archive_enhanced_clean.reindex_axis(index, axis=1)

In [ ]:
# 通过 witter_archive_enhanced.tweet_id 和 df_tweet.id关联创建新的表格twitter_archive_master.csv
twitter_archive_master = pd.merge(witter_archive_enhanced_clean, df_tweet_clean,
                            on=['tweet_id'], how='left')
twitter_archive_master.info()
twitter_archive_master

In [ ]:
# 把数据保存到 twitter_archive_master.csv
twitter_archive_master.to_csv('twitter_archive_master.csv', index=False)